<a href="https://colab.research.google.com/github/zambbo/BERT-blog-sentiment-analysis/blob/master/labeling_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np

In [ ]:
device = torch.device('cuda:0')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#데이터 셋을 BERT모델에 알맞은 형태로 변경
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
    
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    
    def __len__(self):
        return len(self.labels)

In [ ]:
#BERT 모델
#sentiment predict를 위해서는 num_classes를 3으로 설정해준다
#objective/subjective predict를 위해서는 num_classes를 2로 설정해준다.
#default는 3으로 되어있다
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
#KoBERT 모델과 KoBERT vocabulary를 다운받는다.
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

In [ ]:
sentiment_model = BERTClassifier(bertmodel, num_classes=3, dr_rate=0.5).to(device)
subjective_model = BERTClassifier(bertmodel, num_classes=2, dr_rate=0.5).to(device)

In [ ]:
sent_path = '/content/drive/MyDrive/dataset/sentiment_model.pt'
subj_path = '/content/drive/MyDrive/dataset/subjetive_model.pt'

In [ ]:
sentiment_model.load_state_dict(torch.load(sent_path))
subjective_model.load_state_dict(torch.load(subj_path))

In [ ]:
sentiment_model.eval()
subjective_model.eval()

In [ ]:
import pandas as pd

labeling_file_path = '/content/drive/MyDrive/dataset/미니어처.csv'
df = pd.read_csv(labeling_file_path ,index_col=0)
df.sample(3)

In [ ]:
!pip install kss

In [ ]:
# import re

# def preprocessing_sentence(sentence):
#     sent = re.sub("[^ ㄱ-ㅎ가-힣a-zA-Z0-9]", "", sentence)
#     return sent

In [ ]:
# posts = [preprocessing_sentence(post) for post in df['Post'].values]

In [ ]:
# import kss
# post_sentences = []

# for post in posts:
#     sents = [sent for sent in kss.split_sentences(post)]
#     post_sentences.append(sents)
# post_sentences[0]

In [ ]:
# after_labeling_file_path = '/content/drive/MyDrive/dataset/labeling_미니어처.txt'
# with open(after_labeling_file_path,"w",encoding="utf-8") as f:
#     for sents in post_sentences:
#         for sent in sents:
#             f.write(f"{sent}\t1\t1\n")

In [ ]:
prelabeling_data_file_path = '/content/drive/MyDrive/dataset/labeling_미니어처.txt'
sentences = []
df = pd.read_csv(prelabeling_data_file_path, sep='\t', names=['sentence','sentiment','objective'])
df.head()

In [ ]:
sentences = df['sentence'].values

In [ ]:
max_len = 64
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
def sentence2bertdataset(sentences):
    ret_sentences = []
    for sentence in sentences:
        ret_sentences.append([sentence, 1])
    return ret_sentences
sentences = sentence2bertdataset(sentences)

In [ ]:
sentences[0]

In [ ]:
bertdataset = BERTDataset(sentences, 0, 1, tok, max_len, True, False)

In [ ]:
bertdataset[0]

In [ ]:
def bertToTorch(_data, device):
    token_ids, valid_length, segment_ids, label = _data
    token_ids = torch.tensor([token_ids])
    segment_ids = torch.tensor([segment_ids])
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = torch.IntTensor([valid_length.item()])
    return token_ids, valid_length, segment_ids
bertToTorch(bertdataset[0],device)

In [ ]:
def labeling(_bertdataset, _model, score_func):
    predicted_labels = []
    for bs in _bertdataset:
        token_ids, valid_length, segment_ids = bertToTorch(bs, device)
        predicted = _model(token_ids, valid_length, segment_ids)
        predicted_value = score_func(predicted)
        predicted_labels.append(predicted_value)
    
    predicted_labels = np.array(predicted_labels)
    return predicted_labels

In [ ]:
def scoring_func(_predicted):
    predicted = _predicted.cpu().detach().numpy()
    predicted_value = np.argmax(predicted)
    return predicted_value


In [ ]:
sentiment_labels = labeling(bertdataset, sentiment_model, scoring_func)
sentiment_labels[:5]

In [ ]:
subjective_labels = labeling(bertdataset, subjective_model, scoring_func)
subjective_labels[:5]

In [ ]:
after_labeling_data_file_path = '/content/drive/MyDrive/dataset/labeled_미니어처.txt'
sentences = df['sentence'].values
with open(after_labeling_data_file_path, "w", encoding="utf-8") as f:
    for sentence, sentiment, subjective in zip(sentences, sentiment_labels, subjective_labels):
        f.write(f"{sentence}\t{sentiment}\t{subjective}\n")